In [86]:
import csv
import json
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer as CV
import nltk
from IPython.display import display, HTML

In [2]:
with open('stored_variables/sfp_subreddits_by_author.json', 'r') as f:
    sfp_subreddits_by_author = json.load(f)

with open('stored_variables/td_subreddits_by_author.json', 'r') as f:
    td_subreddits_by_author = json.load(f)
    
with open('stored_variables/neither_subreddits_by_author.json', 'r') as f:
    neither_subreddits_by_author = json.load(f)

In [3]:
political_sfp_lst = []

for k in sfp_subreddits_by_author.keys():
    if 'politics' in sfp_subreddits_by_author[k].keys():
        political_sfp_lst.append(sfp_subreddits_by_author[k]['politics'])
        
print(sum(political_sfp_lst))
print(len(political_sfp_lst))

political_td_lst = []

for k in td_subreddits_by_author.keys():
    if 'politics' in td_subreddits_by_author[k].keys():
        political_td_lst.append(td_subreddits_by_author[k]['politics'])
        
print(sum(political_td_lst))
print(len(political_td_lst))

political_neither_lst = []

for k in neither_subreddits_by_author.keys():
    if 'politics' in neither_subreddits_by_author[k].keys():
        political_neither_lst.append(neither_subreddits_by_author[k]['politics'])
        
print(sum(political_neither_lst))
print(len(political_neither_lst))

115414
1289
65378
1087
7971
346


In [8]:
sfp_data = pd.read_csv("../sample_data/sfp_langsample_v4.csv")

In [9]:
td_data = pd.read_csv("../sample_data/td_langsample_v4.csv")

In [16]:
politics_data = pd.read_csv("../sample_data/politics_sample1.csv", 
                            header=None, 
                            names=['author','subreddit','body','score', 'created_dt', 'rand'])

In [17]:
print(sfp_data.columns)
print(td_data.columns)
print(politics_data.columns)

Index(['author', 'subreddit', 'body', 'created_utc', 'created_dt'], dtype='object')
Index(['author', 'subreddit', 'body', 'created_utc', 'created_dt'], dtype='object')
Index(['author', 'subreddit', 'body', 'score', 'created_dt', 'rand'], dtype='object')


In [102]:
politics_text = politics_data.loc[politics_data['body'].notna()]['body'].values

In [34]:
sfp_authors = sfp_data['author'].unique()
td_authors = td_data['author'].unique()
print(len(sfp_authors))
print(len(td_authors))

2000
2000


In [33]:
print(np.sum(sfp_data['subreddit'] == 'SandersForPresident'))
print(np.sum(sfp_data['subreddit'] == 'The_Donald'))
print(np.sum(td_data['subreddit'] == 'SandersForPresident'))
print(np.sum(td_data['subreddit'] == 'The_Donald'))

82916
1885
4476
62877


In [48]:
sfp_posts = sfp_data.loc[sfp_data['subreddit'] == 'SandersForPresident']
td_posts = td_data.loc[td_data['subreddit'] == 'The_Donald']

In [51]:
sfp_min_times = [min(sfp_posts.loc[sfp_posts['author'] == auth]['created_utc']) for auth in sfp_authors]

In [52]:
td_min_times = [min(td_posts.loc[td_posts['author'] == auth]['created_utc']) for auth in td_authors]

In [57]:
sfp_politics = sfp_data.loc[sfp_data['subreddit'] == 'politics']
td_politics = td_data.loc[td_data['subreddit'] == 'politics']
print(sfp_politics.shape)
print(td_politics.shape)

(204584, 5)
(82119, 5)


In [60]:
sum(sfp_politics.loc[sfp_politics['author'] == sfp_authors[0]]['created_utc'] < sfp_min_times[0])
sum(sfp_politics.loc[sfp_politics['author'] == sfp_authors[0]]['created_utc'] > sfp_min_times[0])

26

In [61]:
len(sfp_politics)

204584

In [77]:
sfp_politics_before = pd.DataFrame([], columns = sfp_politics.columns)
sfp_politics_after = pd.DataFrame([], columns = sfp_politics.columns)

for i in range(len(sfp_authors)):
    curr_auth_posts = sfp_politics.loc[sfp_politics['author'] == sfp_authors[i]]
    
    if len(curr_auth_posts) > 0:
        sfp_politics_before = \
sfp_politics_before.append(curr_auth_posts.loc[curr_auth_posts['created_utc'] < sfp_min_times[i]])
        
        sfp_politics_after = \
sfp_politics_after.append(curr_auth_posts.loc[curr_auth_posts['created_utc'] > sfp_min_times[i]])
    

In [79]:
sfp_politics_before

,author,subreddit,body,created_utc,created_dt
857,english_guy_78,politics,Those commas can be tricky little buggers to s...,1193068068,2007-10-22 11:47:48-04
864,english_guy_78,politics,Polls seem to suggest more people than not sup...,1193261879,2007-10-24 17:37:59-04
867,english_guy_78,politics,"Errr, Cheney is who he wants to impeach...",1193344024,2007-10-25 16:27:04-04
897,english_guy_78,politics,"As opposed to the US, where the nut squads alr...",1193764926,2007-10-30 13:22:06-04
929,english_guy_78,politics,"Yep, ironically one could argue that the holoc...",1194565931,2007-11-08 18:52:11-05
...,...,...,...,...,...
3916120,DeserveDonuts,politics,"Oh sorry, I mixed it up with benzos for some r...",1453101565,2016-01-18 02:19:25-05
3916177,DeserveDonuts,politics,"I am that person, and I've never sold or given...",1453102670,2016-01-18 02:37:50-05
3916277,DeserveDonuts,politics,&gt;you're supposed to be taking them steadily...,1453105737,2016-01-18 03:28:57-05
3315277,_IAlwaysLie,politics,"It's the ""You realize, right?"" It builds up th...",1440562655,2015-08-26 00:17:35-04


In [78]:
sfp_politics_after

,author,subreddit,body,created_utc,created_dt
3178299,english_guy_78,politics,"""The legislation would raise the minimum wage ...",1437446254,2015-07-20 22:37:34-04
3178330,english_guy_78,politics,I don't think this really adds up. Who would b...,1437446891,2015-07-20 22:48:11-04
3178426,english_guy_78,politics,Isn't that an argument FOR a higher minimum wa...,1437448905,2015-07-20 23:21:45-04
3178459,english_guy_78,politics,But your initial point was that without Walmar...,1437449647,2015-07-20 23:34:07-04
3178477,english_guy_78,politics,No your point was that these workers would be ...,1437450137,2015-07-20 23:42:17-04
...,...,...,...,...,...
4589605,DeserveDonuts,politics,why is democratically-instituted socialism ins...,1465339497,2016-06-07 18:44:57-04
4589652,DeserveDonuts,politics,that is a very simplistic answer; i don't supp...,1465340382,2016-06-07 18:59:42-04
4291577,_IAlwaysLie,politics,"I think the answer is nationally-funded, not p...",1459575788,2016-04-02 01:43:08-04
4304296,_IAlwaysLie,politics,Sanders estimates that the raised taxes will b...,1459820217,2016-04-04 21:36:57-04


In [80]:
td_politics_before = pd.DataFrame([], columns = td_politics.columns)
td_politics_after = pd.DataFrame([], columns = td_politics.columns)

for i in range(len(td_authors)):
    curr_auth_posts = td_politics.loc[td_politics['author'] == td_authors[i]]
    
    if len(curr_auth_posts) > 0:
        td_politics_before = \
td_politics_before.append(curr_auth_posts.loc[curr_auth_posts['created_utc'] < td_min_times[i]])
        
        td_politics_after = \
td_politics_after.append(curr_auth_posts.loc[curr_auth_posts['created_utc'] > td_min_times[i]])
    

In [81]:
td_politics_before

,author,subreddit,body,created_utc,created_dt
3251,berlinbrown,politics,Dan Rather has a cool reporter voice.,1186977525,2007-08-12 23:58:45-04
3437,berlinbrown,politics,"ick, that isn't a cool story or the Tillman on...",1187721531,2007-08-21 14:38:51-04
3445,berlinbrown,politics,In all due respect. And I am wwwaaaay more li...,1187739659,2007-08-21 19:40:59-04
3487,berlinbrown,politics,I always wonder how people of reddit view cons...,1187917284,2007-08-23 21:01:24-04
3489,berlinbrown,politics,Anybody think it wasn't intentional?,1187917925,2007-08-23 21:12:05-04
...,...,...,...,...,...
3226400,MyWifeDontKnowItsMe,politics,How about you show me evidence that prohibitin...,1452132743,2016-01-06 21:12:23-05
3230507,MyWifeDontKnowItsMe,politics,That's the claim you have been making this ent...,1452217872,2016-01-07 20:51:12-05
3236723,MyWifeDontKnowItsMe,politics,You claimed that ownership and investment are ...,1452375380,2016-01-09 16:36:20-05
3304943,MyWifeDontKnowItsMe,politics,Now I'm definitely voting for Cruz.,1453938614,2016-01-27 18:50:14-05


In [82]:
td_politics_after

,author,subreddit,body,created_utc,created_dt
3442549,berlinbrown,politics,Trump is an amazing showman. He is winning be...,1457057276,2016-03-03 21:07:56-05
3442637,berlinbrown,politics,"Kasich giving no fucks, he is just putting him...",1457058611,2016-03-03 21:30:11-05
3442642,berlinbrown,politics,"OH OK, no clap",1457058721,2016-03-03 21:32:01-05
3442648,berlinbrown,politics,I think they are dancing for the camera.,1457058824,2016-03-03 21:33:44-05
3442668,berlinbrown,politics,"""Trump, Trump, fucking gave to Clinton for bus...",1457059105,2016-03-03 21:38:25-05
...,...,...,...,...,...
3928339,MyWifeDontKnowItsMe,politics,Sounds to me like [she really liked](https://m...,1467154463,2016-06-28 18:54:23-04
3928345,MyWifeDontKnowItsMe,politics,Obviously this is false. She didn't actually d...,1467154523,2016-06-28 18:55:23-04
3928376,MyWifeDontKnowItsMe,politics,Still supported Hillary and Hillary continues ...,1467154964,2016-06-28 19:02:44-04
3928675,MyWifeDontKnowItsMe,politics,When you have to hinge your logic on the notio...,1467160052,2016-06-28 20:27:32-04


In [91]:
#sanity check
display(sfp_politics.loc[sfp_politics['author'] == sfp_authors[0]])
display(sfp_posts.loc[sfp_data['author'] == sfp_authors[0]])
display(sfp_politics_before.loc[sfp_politics_before['author'] == sfp_authors[0]])
display(sfp_politics_after.loc[sfp_politics_after['author'] == sfp_authors[0]])

,author,subreddit,body,created_utc,created_dt
857,english_guy_78,politics,Those commas can be tricky little buggers to s...,1193068068,2007-10-22 11:47:48-04
864,english_guy_78,politics,Polls seem to suggest more people than not sup...,1193261879,2007-10-24 17:37:59-04
867,english_guy_78,politics,"Errr, Cheney is who he wants to impeach...",1193344024,2007-10-25 16:27:04-04
897,english_guy_78,politics,"As opposed to the US, where the nut squads alr...",1193764926,2007-10-30 13:22:06-04
929,english_guy_78,politics,"Yep, ironically one could argue that the holoc...",1194565931,2007-11-08 18:52:11-05
...,...,...,...,...,...
3644263,english_guy_78,politics,"Good. While terrible, it doesn't have much to ...",1447556702,2015-11-14 22:05:02-05
3644285,english_guy_78,politics,They were asked for their opening statement ab...,1447556962,2015-11-14 22:09:22-05
3934051,english_guy_78,politics,None of the other states have been polled in w...,1453443218,2016-01-22 01:13:38-05
4244011,english_guy_78,politics,They hardly started counting yet. It won't eve...,1458710921,2016-03-23 01:28:41-04


,author,subreddit,body,created_utc,created_dt
3159698,english_guy_78,SandersForPresident,$50/mo here. Just threw an extra $50 in as we...,1436987828,2015-07-15 15:17:08-04
3162437,english_guy_78,SandersForPresident,It's about spending the money in the right pla...,1437060156,2015-07-16 11:22:36-04
3427212,english_guy_78,SandersForPresident,Party affiliation: democrat + independent = Hi...,1443055183,2015-09-23 20:39:43-04
4084138,english_guy_78,SandersForPresident,"If the entrance polls are correct, Clinton nee...",1456002132,2016-02-20 16:02:12-05
4084157,english_guy_78,SandersForPresident,"I'm not saying she can't. I'm just saying, lo...",1456002379,2016-02-20 16:06:19-05
...,...,...,...,...,...
4412476,english_guy_78,SandersForPresident,It's irrelevant it's about fracking. It's the...,1461714531,2016-04-26 19:48:51-04
4412481,english_guy_78,SandersForPresident,It's irrelevant it's about fracking. It's the ...,1461714557,2016-04-26 19:49:17-04
4443761,english_guy_78,SandersForPresident,Watching on MSNBC. They talk all day about co...,1462324418,2016-05-03 21:13:38-04
4503393,english_guy_78,SandersForPresident,Anyone have exit polls? It's pretty interesti...,1463534519,2016-05-17 21:21:59-04


,author,subreddit,body,created_utc,created_dt
857,english_guy_78,politics,Those commas can be tricky little buggers to s...,1193068068,2007-10-22 11:47:48-04
864,english_guy_78,politics,Polls seem to suggest more people than not sup...,1193261879,2007-10-24 17:37:59-04
867,english_guy_78,politics,"Errr, Cheney is who he wants to impeach...",1193344024,2007-10-25 16:27:04-04
897,english_guy_78,politics,"As opposed to the US, where the nut squads alr...",1193764926,2007-10-30 13:22:06-04
929,english_guy_78,politics,"Yep, ironically one could argue that the holoc...",1194565931,2007-11-08 18:52:11-05
...,...,...,...,...,...
2175400,english_guy_78,politics,Many of the things the allies did in WWII woul...,1406732325,2014-07-30 10:58:45-04
2590789,english_guy_78,politics,And be replaced by a better system.,1418046484,2014-12-08 08:48:04-05
3141065,english_guy_78,politics,"Yes, he wants to introduce more tax brackets a...",1436539325,2015-07-10 10:42:05-04
3141436,english_guy_78,politics,Taxes are already the same for everyone - ever...,1436544840,2015-07-10 12:14:00-04


,author,subreddit,body,created_utc,created_dt
3178299,english_guy_78,politics,"""The legislation would raise the minimum wage ...",1437446254,2015-07-20 22:37:34-04
3178330,english_guy_78,politics,I don't think this really adds up. Who would b...,1437446891,2015-07-20 22:48:11-04
3178426,english_guy_78,politics,Isn't that an argument FOR a higher minimum wa...,1437448905,2015-07-20 23:21:45-04
3178459,english_guy_78,politics,But your initial point was that without Walmar...,1437449647,2015-07-20 23:34:07-04
3178477,english_guy_78,politics,No your point was that these workers would be ...,1437450137,2015-07-20 23:42:17-04
3178516,english_guy_78,politics,If you don't believe that Walmart creates the ...,1437450855,2015-07-20 23:54:15-04
3178525,english_guy_78,politics,Surely this just highlights that the other job...,1437450969,2015-07-20 23:56:09-04
3178558,english_guy_78,politics,But then by your logic ALL employers regardles...,1437451498,2015-07-21 00:04:58-04
3178582,english_guy_78,politics,Clearly it follows pay for all jobs will incre...,1437451804,2015-07-21 00:10:04-04
3178632,english_guy_78,politics,Pay must increase to motivate people to do har...,1437452766,2015-07-21 00:26:06-04


In [109]:
sfp_politics_before_text = sfp_politics_before.loc[sfp_politics_before['body'].notna()]['body'].values
sfp_politics_after_text = sfp_politics_after.loc[sfp_politics_after['body'].notna()]['body'].values

td_politics_before_text = td_politics_before.loc[td_politics_before['body'].notna()]['body'].values
td_politics_after_text = td_politics_after.loc[td_politics_after['body'].notna()]['body'].values

sfp_posts_text = sfp_posts.loc[sfp_posts['body'].notna()]['body'].values
td_posts_text = td_posts.loc[td_posts['body'].notna()]['body'].values

In [105]:
filenames = ['politics_train.txt', 
             'sfp_politics_before.txt', 
             'sfp_politics_after.txt',
             'td_politics_before.txt',
             'td_politics_after.txt',
             'sfp_posts.txt',
             'td_posts.txt']

text_arrs = [politics_text,
             sfp_politics_before_text,
             sfp_politics_after_text,
             td_politics_before_text,
             td_politics_after_text,
             sfp_posts_text,
             td_posts_text]

train_arrs = []

for i in range(len(text_arrs)):
    lm_train_arr = \
[(' '.join(nltk.word_tokenize(s)).lower() + '\n') for v in text_arrs[i] for s in nltk.sent_tokenize(v)]
    train_arrs.append(lm_train_arr)

In [111]:
train_arrs[4] = \
[(' '.join(nltk.word_tokenize(s)).lower() + '\n') for v in td_politics_after_text for s in nltk.sent_tokenize(v)]


In [112]:
for i in range(len(train_arrs)):
    print(train_arrs[i][:10])
    print("\n\n")

['true .\n', "but the ***only*** way that is likely to occur is to start rubbing the general public 's noses in the way that the 1 % have already stolen most of their country\n", 'fuck him , right ?\n', 'bernie or bust !\n', 'cnn did a map breakdown last night where they gave bernie a win in every remaining state and guess what ... ..\n', 'http : //imgur.com/neqdsrt\n', 'i disagree .\n', "the entire race basically every mainstream media outlet has shown the delegate count including super delegates giving clinton the appearance of being like over 4-5 hundred delegates in the lead when it 's much closer to like 100-250 .\n", 'you may think this has no impact , but i would imagine there is some slight impact based on this .\n', 'some studies have shown that voters are much more inclined to choose a candidate if they believe they are definitely going to win the nomination .\n']



['those commas can be tricky little buggers to spot sometimes , but they serve a very useful purpose ...\n', '

In [113]:
for i in range(len(train_arrs)):
    lm_train_file = open(filenames[i], 'w+')
    lm_train_file.writelines(train_arrs[i])
    lm_train_file.close()

In [114]:
for i in range(len(train_arrs)):
    print(len(train_arrs[i]))

28251
356705
262296
182212
54922
216601
129730
